## Load necessary modules

In [1]:
# show images inline
# %matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
# from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
# import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this to change which GPU to use
gpu = 1

# set the modified tf session as backend in keras
#setup_gpu(gpu)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

Using TensorFlow backend.


## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
# model_path = os.path.join('..', 'snapshots', 'forward_inference.h5')
model_path = './converted_model.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
# model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'b', 1: 'u'} ##unused-->  , 2: 'g', 3: 'c', 4: 'bi', 5: 'h'}

tracking <tf.Variable 'Variable:0' shape=(21, 4) dtype=float32, numpy=
array([[-33.504   ,  -8.376   ,  33.504   ,   8.376   ],
       [-41.536   , -10.384   ,  41.536   ,  10.384   ],
       [-52.8     , -13.2     ,  52.8     ,  13.2     ],
       [-26.226274, -10.70032 ,  26.226274,  10.70032 ],
       [-32.513565, -13.265534,  32.513565,  13.265534],
       [-41.330803, -16.862967,  41.330803,  16.862967],
       [-19.004488, -14.766487,  19.004488,  14.766487],
       [-23.560482, -18.306494,  23.560482,  18.306494],
       [-29.949764, -23.270967,  29.949764,  23.270967],
       [-16.752   , -16.752   ,  16.752   ,  16.752   ],
       [-20.768   , -20.768   ,  20.768   ,  20.768   ],
       [-26.4     , -26.4     ,  26.4     ,  26.4     ],
       [-14.760761, -19.01186 ,  14.760761,  19.01186 ],
       [-18.299395, -23.56962 ,  18.299395,  23.56962 ],
       [-23.261944, -29.961382,  23.261944,  29.961382],
       [-10.704646, -26.215677,  10.704646,  26.215677],
       [-13.27089

/home/myid/ea51366/anaconda3/envs/faster/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
def getListOfFiles(dirName,exceptions):
    l=list()
    l = os.listdir(dirName)
    output = list()
    for f in l:
        if f.split('.')[-1] not in exceptions:
            continue

        output.append(str(dirName)+'/'+str(f))

    return output


# ooo = getListOfFiles('../../../thesis/all_data/Right/0150_1_B/rgb/')
# ooo = getListOfFiles('./data_for_conversion/img4/')

# ooo = getListOfFiles('./Yolo_Images')

# print(ooo[:])

# for f in ooo:
#     if f[0]=='f' and f[1]=='r':
#         os.rename('./data_for_conversion/img1/'+str(f),'./data_for_conversion/img1/'+str(f)+'.jpg')


In [ ]:
# path1 = './sanet_dataset/1'
# path2 = './sanet_dataset/2'
# path3 = './sanet_dataset/3'
# path4 = './sanet_dataset/4'
# path5 = './sanet_dataset/5'

# listOfFiles1=getListOfFiles(path1,['jpg'])
# listOfFiles2=getListOfFiles(path2,['jpg'])
# listOfFiles3=getListOfFiles(path3,['jpg'])
# listOfFiles4=getListOfFiles(path4,['jpg'])
# listOfFiles5=getListOfFiles(path5,['jpg'])

# listOfFiles = list()
# listOfFiles.extend(listOfFiles1)
# listOfFiles.extend(listOfFiles2)
# listOfFiles.extend(listOfFiles3)
# listOfFiles.extend(listOfFiles4)
# listOfFiles.extend(listOfFiles5)

# print (len(listOfFiles))
# print (listOfFiles[:10])

# ooo = listOfFiles
# ooo.sort()

In [5]:
path = './image'

listOfFiles=getListOfFiles(path,['jpg'])

print (len(listOfFiles))
print (listOfFiles[:10])

ooo = listOfFiles
ooo.sort()

50
['./image/frame0049.jpg', './image/frame0043.jpg', './image/frame0001.jpg', './image/frame0016.jpg', './image/frame0003.jpg', './image/frame0011.jpg', './image/frame0038.jpg', './image/frame0026.jpg', './image/frame0005.jpg', './image/frame0025.jpg']


## Run detection on example

def labelImage(imgDir,f):
    # load image
    image = read_image_bgr(imgDir)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        
        printToFile = ""
        printToFile += str(imgDir)
        printToFile += ","
        printToFile += str(int(round(box[0])))
        printToFile += ","
        printToFile += str(int(round(box[1])))
        printToFile += ","
        printToFile += str(int(round(box[2])))
        printToFile += ","
        printToFile += str(int(round(box[3])))
        printToFile += ","
        printToFile += str(labels_to_names[label])
        printToFile += '\n'
        f.write(printToFile)
        



f=open("allLabels.txt", "a+")
counter = 0
for i in listOfFiles3:
    labelImage(str(i),f)
    counter +=1
    if counter%100 == 0:
        print(counter)
f.close()

f=open("guru99.txt", "r")
if f.mode == 'r':
    contents =f.read()
    for c in contents:
        print(c)
f.close()

In [11]:
####################################################
#### Generate yolo labels
####################################################


# Use to depict imput test image
counter = 1;
def showImage(imgDir):
    # load image
    image = read_image_bgr(imgDir)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image,min_side=1080,max_side=1920)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
#     print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    prev_labels = []
    
    picked = False
    bin_seen = False
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        print (label,score,box)
#         if label == 0 or label =='0':
#             if box[-1] < 700:
#                 if score < 0.62 or picked==True:
#                     continue
#             elif score<0.9:
#                 continue
#         if label == 1 or label == '1':
#             if box[0]>1000:
#                 if score <0.97:
#                     continue
#             if box[-1] < 700:
#                 if score < 0.62 or picked:
#                     continue
#             elif score < 0.93:
#                 continue
# #             elif score 
#         if label == 3 or label == '3':
#             box[0] = 0
#             box[1] = 808
#             box[2] = 1773
#             box[3] = 1023
# #             if score<0.6:
# #                 continue
#         if score < 0.5 and label != 4:
#             break
#         if bin_seen and label == 4:
#             break
#         if score < 0.45:
#             break
        
#         if label == 4:
#             bin_seen = True
#         if box[-1]<700:
#             picked=True
        
        if score < 0.5:
            break
    
        prev_labels.append(label)
        
        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        
        printToFile = ""
        printToFile += str(imgDir)
        printToFile += ","
        printToFile += str(int(round(box[0])))
        printToFile += ","
        printToFile += str(int(round(box[1])))
        printToFile += ","
        printToFile += str(int(round(box[2])))
        printToFile += ","
        printToFile += str(int(round(box[3])))
        printToFile += ","
        printToFile += str(labels_to_names[label])
        printToFile += '\n'
#         print (imgDir,box[0],box[1],box[2],box[3],labels_to_names[label])
#         print (printToFile)
        f = open("yolo_labels_6_15_21.txt", "a")
        f.write(printToFile)
        f.close()
        if counter%100 == 0:
            print(counter)
#         print (score)
        
#     plt.figure(figsize=(15, 15))
#     plt.axis('off')
#     plt.imshow(draw)
#     plt.show()
#     im_name = imgDir.split('/')[-1].split('.')[0]
#     global counter
#     out_dir = './detected_new/'+im_name+'_'+str(counter)+"_detected.jpg"
#     counter += 1
#     draw = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
#     cv2.imwrite(out_dir, draw)
   
# showImage('./data_for_conversion/img4/frame4480.jpg')

In [7]:
# ooo = listOfFiles
# ooo.sort()
now = time.time()
for fn in ooo:
    showImage2(fn)
now2 = time.time()

print(now2-now)

processing time:  4.245896816253662
1 0.9999988 [ 939.88635  820.4262  1076.7599   942.56586]
./image/frame0000.jpg,940,820,1077,943,u
0.9999988
0 0.99999857 [542.3775  799.32874 665.1593  912.2976 ]
./image/frame0000.jpg,542,799,665,912,b
0.99999857
1 0.99999344 [ 28.17877 809.9896  215.50638 966.1744 ]
./image/frame0000.jpg,28,810,216,966,u
0.99999344
0 0.9999931 [751.7568  753.76917 886.7509  873.553  ]
./image/frame0000.jpg,752,754,887,874,b
0.9999931
0 0.9999814 [321.66183 760.3874  465.1465  892.65326]
./image/frame0000.jpg,322,760,465,893,b
0.9999814
0 0.9999112 [1452.5065   788.1713  1610.5333   932.57184]
./image/frame0000.jpg,1453,788,1611,933,b
0.9999112
1 0.95793545 [1157.7052   756.82715 1290.2726   875.93713]
./image/frame0000.jpg,1158,757,1290,876,u
0.95793545
0 0.81688255 [1156.9896  757.2395 1291.1361  875.9647]
processing time:  0.093780517578125
0 0.99999905 [540.9417 799.4386 665.2562 912.7829]
./image/frame0001.jpg,541,799,665,913,b
0.99999905
1 0.99999905 [ 940.05

processing time:  0.09268569946289062
1 0.9999987 [554.66315 754.3928  689.00476 876.81464]
./image/frame0011.jpg,555,754,689,877,u
0.9999987
1 0.99999726 [617.59106 802.35724 776.6773  957.8345 ]
./image/frame0011.jpg,618,802,777,958,u
0.99999726
0 0.99999654 [752.9122  756.41144 885.1373  875.482  ]
./image/frame0011.jpg,753,756,885,875,b
0.99999654
1 0.9999956 [ 940.40106  824.0009  1072.2805   943.01074]
./image/frame0011.jpg,940,824,1072,943,u
0.9999956
0 0.9999944 [1177.2239  820.1608 1313.1256  941.7574]
./image/frame0011.jpg,1177,820,1313,942,b
0.9999944
0 0.99996257 [1004.71594  767.6758  1133.8271   888.5475 ]
./image/frame0011.jpg,1005,768,1134,889,b
0.99996257
0 0.9999269 [1308.095    753.92737 1441.6635   877.45154]
./image/frame0011.jpg,1308,754,1442,877,b
0.9999269
1 0.98557615 [590.11053 782.9236  742.42786 920.4991 ]
./image/frame0011.jpg,590,783,742,920,u
0.98557615
0 0.298633 [ 940.95953  815.5205  1082.7402   943.7597 ]
processing time:  0.10450625419616699
0 0.9999

processing time:  0.09406042098999023
1 0.99999845 [712.44995 831.82416 850.51733 958.4279 ]
./image/frame0022.jpg,712,832,851,958,u
0.99999845
0 0.9999974 [548.62274 762.0197  681.95667 881.8645 ]
./image/frame0022.jpg,549,762,682,882,b
0.9999974
1 0.99999666 [232.59514 787.0579  393.80606 925.1372 ]
./image/frame0022.jpg,233,787,394,925,u
0.99999666
1 0.99999475 [431.49454 803.39325 592.12854 949.3356 ]
./image/frame0022.jpg,431,803,592,949,u
0.99999475
0 0.999992 [1178.3851  821.2051 1314.614   942.8613]
./image/frame0022.jpg,1178,821,1315,943,b
0.999992
0 0.9999838 [ 999.5844   764.2112  1134.4714   893.71063]
./image/frame0022.jpg,1000,764,1134,894,b
0.9999838
0 0.99961543 [1306.9851   751.28143 1442.1886   876.4355 ]
./image/frame0022.jpg,1307,751,1442,876,b
0.99961543
0 0.42675874 [830.8642  268.47043 941.5094  382.12857]
processing time:  0.0935823917388916
1 0.99999917 [232.58122 784.2311  394.83295 920.42865]
./image/frame0023.jpg,233,784,395,920,u
0.99999917
0 0.99999845 [54

processing time:  0.09285926818847656
1 0.9999987 [429.53116 800.5952  592.36896 948.6967 ]
./image/frame0033.jpg,430,801,592,949,u
0.9999987
1 0.99999785 [713.2085  830.7902  850.5736  959.85236]
./image/frame0033.jpg,713,831,851,960,u
0.99999785
1 0.99999535 [233.07834 783.63055 396.31317 923.174  ]
./image/frame0033.jpg,233,784,396,923,u
0.99999535
0 0.9999906 [1177.5613  819.9874 1314.5568  944.0407]
./image/frame0033.jpg,1178,820,1315,944,b
0.9999906
0 0.9999856 [ 997.7195   762.8236  1135.7067   896.73517]
./image/frame0033.jpg,998,763,1136,897,b
0.9999856
0 0.9999728 [2.35535383e-01 3.32324158e+02 1.09812775e+02 4.71098297e+02]
./image/frame0033.jpg,0,332,110,471,b
0.9999728
0 0.99919695 [1307.5247   754.9347  1441.7349   875.09186]
./image/frame0033.jpg,1308,755,1442,875,b
0.99919695
1 0.25321794 [1305.2205   753.0767  1442.166    874.26697]
processing time:  0.09447503089904785
1 0.9999982 [712.36176 830.97363 849.17444 958.03613]
./image/frame0034.jpg,712,831,849,958,u
0.9999

processing time:  0.09393763542175293
1 0.99999833 [713.6113 833.5385 850.1296 956.4481]
./image/frame0043.jpg,714,834,850,956,u
0.99999833
1 0.999997 [429.36307 799.9864  590.6988  949.4515 ]
./image/frame0043.jpg,429,800,591,949,u
0.999997
0 0.9999943 [1175.7753  818.9562 1313.4354  944.1922]
./image/frame0043.jpg,1176,819,1313,944,b
0.9999943
1 0.99999404 [234.57365 785.95776 394.98932 921.77295]
./image/frame0043.jpg,235,786,395,922,u
0.99999404
0 0.9999701 [ 999.0788  764.6098 1134.01    896.7419]
./image/frame0043.jpg,999,765,1134,897,b
0.9999701
0 0.9999372 [ 860.3104   440.05716 1035.8218   598.71045]
./image/frame0043.jpg,860,440,1036,599,b
0.9999372
0 0.9996891 [1307.3484   751.87604 1441.0536   876.22394]
./image/frame0043.jpg,1307,752,1441,876,b
0.9996891
0 0.12621206 [1199.9525   805.62524 1349.0525   917.48505]
processing time:  0.09333133697509766
1 0.999998 [712.4801  833.7928  849.09094 959.2757 ]
./image/frame0044.jpg,712,834,849,959,u
0.999998
1 0.99999726 [429.76135

In [8]:
print('done!')

done!


In [ ]:
# from PIL import Image

In [6]:
################################################################################
########## Annotate images (add bounding boxes and labels) #####################
################################################################################

# Use to depict input test image
counter = 1;
def showImage2(imgDir):
    # load image
    image = read_image_bgr(imgDir)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image,min_side=1080,max_side=1920)
#     print(image.shape)
#     print (np.expand_dims(image, axis=0).shape)
#     print (model.predict_on_batch(np.expand_dims(image, axis=0))[0])
#     print (model.predict_on_batch(np.expand_dims(image, axis=0))[0].shape)
#     print (model.predict_on_batch(np.expand_dims(image, axis=0))[1])
#     print (model.predict_on_batch(np.expand_dims(image, axis=0))[1].shape)
#     print (model.predict_on_batch(np.expand_dims(image, axis=0))[2])
#     print (model.predict_on_batch(np.expand_dims(image, axis=0))[2].shape)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    prev_labels = []
    # visualize detections
    picked = False
    bin_seen = False
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        print (label,score,box)
#         if label == 0 or label =='0':
#             if box[-1] < 700:
#                 if score < 0.62 or picked==True:
#                     continue
#             elif score<0.9:
#                 continue
#         if label == 1 or label == '1':
#             if box[0]>1000:
#                 if score <0.97:
#                     continue
#             if box[-1] < 700:
#                 if score < 0.62 or picked:
#                     continue
#             elif score < 0.93:
#                 continue
# #             elif score 
#         if label == 3 or label == '3':
#             box[0] = 0
#             box[1] = 808
#             box[2] = 1773
#             box[3] = 1023
# #             if score<0.6:
# #                 continue
#         if score < 0.5 and label != 4:
#             break
#         if bin_seen and label == 4:
#             break
#         if score < 0.45:
#             break
        
#         if label == 4:
#             bin_seen = True
#         if box[-1]<700:
#             picked=True
        
#         print (label,score,box)
#         if label in prev_labels:
#             continue;
        if score < 0.85:
            break
        prev_labels.append(label)
        
        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        
        printToFile = ""
        printToFile += str(imgDir)
        printToFile += ","
        printToFile += str(int(round(box[0])))
        printToFile += ","
        printToFile += str(int(round(box[1])))
        printToFile += ","
        printToFile += str(int(round(box[2])))
        printToFile += ","
        printToFile += str(int(round(box[3])))
        printToFile += ","
        printToFile += str(labels_to_names[label])
        #print (imgDir,box[0],box[1],box[2],box[3],labels_to_names[label])
        print (printToFile)
        print (score)
        
#     plt.figure(figsize=(15, 15))
#     plt.axis('off')
#     plt.imshow(draw)
#     plt.show()
    im_name = imgDir.split('/')[-1].split('.')[0]
    global counter
#     out_dir = './forward_detected/'+im_name+'_'+str(counter)+"_detected.jpg"
    out_dir = './detected_test2/'+im_name+"_detected.jpg"
    counter += 1
    draw = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_dir, draw)
# ooo.sort()  
# showImage2(ooo[1])

In [ ]:
showImage2('./')

In [ ]:
showImage2(ooo[182])

In [ ]:
showImage2(ooo[400])

In [ ]:
showImage2(ooo[500])

In [ ]:
showImage2(ooo[600])

In [ ]:
showImage2(ooo[700])

In [ ]:
showImage2(ooo[800])

In [ ]:
showImage2(ooo[900])

In [ ]:
showImage2(ooo[998])